In [20]:
import os
import cv2
import numpy as np
import tkinter as tk
from tkinter import Canvas, Button, Label, font
from PIL import ImageGrab
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
import tkinter as tk
from PIL import Image, ImageTk

Gerekli kütüphanelerin ve modüllerin içe aktarılması: os, cv2, numpy, PIL ve tkinter kütüphaneleri içe aktarılır. Ayrıca, ekran görüntüsü almak için ImageGrab modülü içe aktarılır.

In [21]:
# Sınıf sayısı
num_classes = 10

In [22]:
# Görüntü boyutları
img_width, img_height = 28, 28

In [23]:
def load_data(data_dir):
    images = []
    labels = []

    class_names = os.listdir(data_dir)
    num_classes = len(class_names)
    label_encoder = LabelEncoder()

    for i, class_name in enumerate(class_names):
        class_dir = os.path.join(data_dir, class_name)

        if os.path.isdir(class_dir):
            label = i
            label_encoder.fit([label])
            
            for filename in os.listdir(class_dir):
                image_path = os.path.join(class_dir, filename)
                image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                image = cv2.resize(image, (img_width, img_height))
                images.append(image)
                labels.append(label)

    images = np.array(images)
    labels = np.array(labels)

    return images, labels, num_classes


Veri setinin yüklenmesi: load_data fonksiyonu kullanılarak belirtilen veri dizininden görüntüler ve etiketler yüklenir. Veriler, images ve labels adlı numpy dizileri olarak döndürülür.

In [24]:

# Veri setini yükle
data_dir = 'C:/Users/ysfcl\Desktop/YapayzekaProje/binary_images'
images, labels, num_classes = load_data(data_dir)

In [25]:
# Veriyi eğitim ve test veri setlerine ayırma
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

Veri setinin eğitim ve test verilerine ayrılması: train_test_split fonksiyonu kullanılarak yüklenen veri seti eğitim ve test veri setlerine ayrılır. Eğitim ve test verileri, x_train, x_test, y_train ve y_test adlı numpy dizileri olarak elde edilir.

In [26]:
from tensorflow.keras.utils import to_categorical

# y_train'i one-hot encoding'e dönüştürme
y_train_encoded = to_categorical(y_train, num_classes)

# y_test'i one-hot encoding'e dönüştürme
y_test_encoded = to_categorical(y_test, num_classes)


Veri önişleme: Eğitim ve test verileri normalize edilir ve etiketler one-hot encoding'e dönüştürülür.

In [27]:
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print("x_test shape:", x_test.shape)
print("y_test shape:", y_test.shape)


x_train shape: (252, 28, 28)
y_train shape: (252,)
x_test shape: (63, 28, 28)
y_test shape: (63,)


In [28]:
# Veriyi train ve test olarak ayır
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.1, random_state=42)


In [29]:
# Veriyi normalize etme
x_train = x_train / 255.0
x_test = x_test / 255.0

Verileri normalize etmek, değerlerin 0 ile 1 arasında bir ölçekte olmasını sağlar. Bu genellikle görüntü verileriyle çalışırken kullanılan bir ön işleme adımıdır.

x_train ve x_test verilerini 0 ile 255 arasındaki piksel değerlerine sahip görüntüler içeriyorsa, bu kod satırları verileri 0 ile 1 arasında bir ölçekte normalleştirir. Bunu yaparak, modelin daha iyi öğrenme yapmasını ve daha iyi sonuçlar elde etmesini sağlamayı hedefledik.

In [30]:
# Etiketleri one-hot encoding ile dönüştürme
label_binarizer = LabelBinarizer()
y_train = label_binarizer.fit_transform(y_train)
y_test = label_binarizer.transform(y_test)


In [31]:
# Modeli oluşturma
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

Modelin oluşturulması: Sequential modeli kullanılarak bir sinir ağı modeli oluşturulur. Model, 2D evrişimli katmanlar, pooling katmanları, tam bağlantılı katmanlar ve dropout katmanı içerir.


In [32]:
# Modeli derleme
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


Modelin derlenmesi: Model, adam optimizeri ve categorical_crossentropy kaybı ile derlenir.

In [33]:
# Modeli eğitme
model.fit(x_train[..., np.newaxis], y_train, epochs=25, batch_size=8, validation_data=(x_test[..., np.newaxis], y_test))


Epoch 1/25
36/36 [==============================] - 1s 12ms/step - loss: 2.3244 - accuracy: 0.0813 - val_loss: 2.3032 - val_accuracy: 0.1562
Epoch 2/25
36/36 [==============================] - 0s 7ms/step - loss: 2.2972 - accuracy: 0.1025 - val_loss: 2.2923 - val_accuracy: 0.0938
Epoch 3/25
36/36 [==============================] - 0s 7ms/step - loss: 2.3127 - accuracy: 0.1237 - val_loss: 2.2904 - val_accuracy: 0.1875
Epoch 4/25
36/36 [==============================] - 0s 6ms/step - loss: 2.2778 - accuracy: 0.1802 - val_loss: 2.2281 - val_accuracy: 0.1562
Epoch 5/25
36/36 [==============================] - 0s 6ms/step - loss: 2.1866 - accuracy: 0.2049 - val_loss: 2.1083 - val_accuracy: 0.1562
Epoch 6/25
36/36 [==============================] - 0s 6ms/step - loss: 2.0722 - accuracy: 0.2297 - val_loss: 1.9553 - val_accuracy: 0.5000
Epoch 7/25
36/36 [==============================] - 0s 7ms/step - loss: 1.8745 - accuracy: 0.3039 - val_loss: 1.7718 - val_accuracy: 0.3750
Epoch 8/25
36/36 [=

Modelin eğitilmesi: Model, eğitim verileriyle belirli bir epoch sayısında eğitilir.

In [34]:
# Modeli değerlendirme
loss, accuracy = model.evaluate(x_test[..., np.newaxis], y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)


1/1 [==============================] - 0s 212ms/step - loss: 1.2328 - accuracy: 0.5625
Test loss: 1.2327985763549805
Test accuracy: 0.5625


Modelin değerlendirilmesi: Model, test verileri üzerinde değerlendirilir ve kayıp değeri ile doğruluk oranı görüntülenir.

In [35]:
# Yeni bir görüntüyü tahmin etme
def predict_new_image(image_path):
    # Görüntüyü yükle
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (img_width, img_height))
    image = image / 255.0
    
    # Tahmin yap
    input_image = image[np.newaxis, ..., np.newaxis]
    prediction = model.predict(input_image)
    predicted_label = np.argmax(prediction)
    
    return predicted_label

Yeni bir görüntünün tahmin edilmesi: predict_new_image fonksiyonu kullanılarak yeni bir görüntünün tahmini yapılır. Görüntü önceden belirtilen boyutlara yeniden boyutlandırılır, normalize edilir ve model üzerinde tahmin işlemi gerçekleştirilir.

In [36]:
# Yeni bir görüntüyü tahmin etme örneği
new_image_path = 'C:/Users/ysfcl/Desktop/YapayzekaProje/numbers/1.jpg'
predicted_label = predict_new_image(new_image_path)
print('Tahmin:', predicted_label)

1/1 [==============================] - 0s 79ms/step
Tahmin: 1
